## Text Preprocessing

In [3]:
# load libraries
import pandas as pd
import nltk # https://www.nltk.org/ ; nltk helps with tokenization, stopwords dictionary, lemmatization
import re # https://docs.python.org/3/library/re.html ; re is used for regular expressions
import numpy as np

brandwatch = pd.read_csv("'Brandwatch_All_noRT.csv'.csv", header = 0, engine='python')
brandwatch.head()



,brand,date,full_text,page_type,Clean_text,Product_category,Sentiment
0,Listerine,12/30/2018,RT @KevGBKNY I think that?s out your system 2-...,twitter,think system days day use lot listerine peroxide,LISTERINEÃƒâ€šÃ‚Â® NIGHTLY RESET,1
1,Listerine,12/30/2018,"I love these strips, but have had trouble find...",review,love strips trouble finding area please tell s...,LISTERINEÃƒâ€šÃ‚Â® Floss Products,1
2,Listerine,12/30/2018,"I brush my teeth with walmart brand listerine,...",forum,brush teeth walmart brand listerine ok,LISTERINEÃƒâ€šÃ‚Â® SENSITIVITY,1
3,Listerine,12/30/2018,RT @chaddukes Sometimes when I watch the Redsk...,twitter,sometimes watch redskins play football like ro...,LISTERINEÃƒâ€šÃ‚Â® On-The-Go Oral Care Products,0
4,Listerine,12/30/2018,Jawn be gettin off work tryna meet up / jawn b...,twitter,jawn gettin work tryna meet jawn b gettin outt...,LISTERINEÃƒâ€šÃ‚Â® NIGHTLY RESET,1


In [53]:
# check the structure of the data: number of rows, columns, type of each column, number of non-null values in each column
brandwatch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240337 entries, 0 to 240336
Data columns (total 7 columns):
brand               240337 non-null object
date                240337 non-null object
full_text           240337 non-null object
page_type           240294 non-null object
Clean_text          240337 non-null object
Product_category    74297 non-null object
Sentiment           240337 non-null int64
dtypes: int64(1), object(6)
memory usage: 12.8+ MB


## Clean up to match Brandwatch

In [56]:
brandwatch['Product_category'].value_counts()

LISTERINEÃƒâ€šÃ‚Â® Antiseptic Mouthwash Collection                                17266
LISTERINEÃƒâ€šÃ‚Â® On-The-Go Oral Care Products                                   14754
LISTERINEÃƒâ€šÃ‚Â® NIGHTLY RESET                                                  11243
LISTERINEÃƒâ€šÃ‚Â® SENSITIVITY                                                     8245
LISTERINEÃƒâ€šÃ‚Â® ZERO alcohol-free Mouthwash Collection                          8065
LISTERINEÃƒâ€šÃ‚Â® ULTRACLEANÃƒâ€šÃ‚Â® Tartar Control Mouthwash Collection         4652
LISTERINEÃƒâ€šÃ‚Â® Floss Products                                                  4245
LISTERINEÃƒâ€šÃ‚Â® SMART RINSEÃƒâ€šÃ‚Â® Kids Mouthwash Collection                  2881
LISTERINEÃƒâ€šÃ‚Â® Fluoride Toothpaste Collection                                  1087
LISTERINEÃƒâ€šÃ‚Â® TOTAL CARE Mouthwash Collection                                  979
LISTERINEÃƒâ€šÃ‚Â® NATURALS Mouthwash Collection                                    723
LISTERINEÃƒâ€šÃ‚Â® HEALTHY WHITE

In [58]:
# Product Collection names to match Bazar
# Clean Product Collection names
new = brandwatch["Product_category"].str.split("® ", n=1, expand = True)
new[1] = new[1].str.replace('SMART RINSEÂ® Kids Mouthwash Collection', 'Kids Mouthwash Collection')
new[1] = new[1].str.replace('ultracleanãƒâ€šã‚â® tartar control mouthwash collection', 'Ultraclean tartar control mouthwash collection')
new[1] = new[1].str.replace('smart rinseãƒâ€šã‚â® kids mouthwash collection', 'Kids Mouthwash Collection')
new[1] = new[1].str.replace('ultracleanÃ¢Â® tartar control mouthwash collection', 'Ultraclean tartar control mouthwash collection')
new[1] = new[1].str.replace('healthy whiteãƒâ¢?ã‚â¢ teeth whitening mouthwash collection', 'teeth whitening mouthwash collection')
new[1] = new[1].str.replace('healthy whiteÃ¢?Â¢ teeth whitening mouthwash collection', 'teeth whitening mouthwash collection')
new[1] = new[1].str.replace('fluoride defenseÃ¢?Â¢', 'fluoride defense')
new[1] = new[1].str.replace('ULTRACLEANÃƒâ€šÃ‚Â® Tartar Control Mouthwash Collection', 'Tartar Control Mouthwash Collection')
new[1] = new[1].str.replace('SMART RINSEÃƒâ€šÃ‚Â® Kids Mouthwash Collection', 'Kids Mouthwash Collection')
new[1] = new[1].str.replace('HEALTHY WHITEÃƒÂ¢?Ã‚Â¢ Teeth Whitening Mouthwash Collection', 'Teeth Whitening Mouthwash Collection')
new[1].value_counts()


Antiseptic Mouthwash Collection                                17266
On-The-Go Oral Care Products                                   14754
NIGHTLY RESET                                                  11243
SENSITIVITY                                                     8245
ZERO alcohol-free Mouthwash Collection                          8065
Tartar Control Mouthwash Collection                             4652
Floss Products                                                  4245
Kids Mouthwash Collection                                       2881
Fluoride Toothpaste Collection                                  1087
TOTAL CARE Mouthwash Collection                                  979
NATURALS Mouthwash Collection                                    723
HEALTHY WHITEÃƒÂ¢?Ã‚Â¢ Teeth Whitening Mouthwash Collection      157
Name: 1, dtype: int64

In [59]:
# bring into df and lower
brandwatch["Product_Collection_New"] = new[1]
brandwatch["Product_category"] = brandwatch["Product_Collection_New"].str.lower()
brandwatch.head(2)

,brand,date,full_text,page_type,Clean_text,Product_category,Sentiment,Product_Collection_New
0,Listerine,12/30/2018,RT @KevGBKNY I think that?s out your system 2-...,twitter,think system days day use lot listerine peroxide,nightly reset,1,NIGHTLY RESET
1,Listerine,12/30/2018,"I love these strips, but have had trouble find...",review,love strips trouble finding area please tell s...,floss products,1,Floss Products


In [60]:
# only retain those column we need
brandwatch_clean = brandwatch.iloc[:,[0,1,2,3,4,5,6]]
brandwatch_clean.head(2)


,brand,date,full_text,page_type,Clean_text,Product_category,Sentiment
0,Listerine,12/30/2018,RT @KevGBKNY I think that?s out your system 2-...,twitter,think system days day use lot listerine peroxide,nightly reset,1
1,Listerine,12/30/2018,"I love these strips, but have had trouble find...",review,love strips trouble finding area please tell s...,floss products,1


In [61]:

#change column names to match Brandwatch
column_mapping = {brandwatch_clean.columns[1]:'Date', brandwatch_clean.columns[2]:'Review_original', brandwatch_clean.columns[4]:'Review_clean', brandwatch_clean.columns[0]:'Brand', brandwatch_clean.columns[6]:'Sentiment', brandwatch_clean.columns[3]:'Source', brandwatch_clean.columns[5]:'Product_category'}
brandwatch_clean.rename(columns = column_mapping, inplace = True)
brandwatch_clean.head(2)

,Brand,Date,Review_original,Source,Review_clean,Product_category,Sentiment
0,Listerine,12/30/2018,RT @KevGBKNY I think that?s out your system 2-...,twitter,think system days day use lot listerine peroxide,nightly reset,1
1,Listerine,12/30/2018,"I love these strips, but have had trouble find...",review,love strips trouble finding area please tell s...,floss products,1


In [62]:
# reorder columns
cols = list(brandwatch_clean.columns.values)
cols

['Brand',
 'Date',
 'Review_original',
 'Source',
 'Review_clean',
 'Product_category',
 'Sentiment']

In [63]:
# reorder columns2
brandwatch_clean = brandwatch_clean[['Date','Brand', 'Product_category', 'Source', 'Review_original', 'Review_clean', 'Sentiment']]
brandwatch_clean.head(2)

,Date,Brand,Product_category,Source,Review_original,Review_clean,Sentiment
0,12/30/2018,Listerine,nightly reset,twitter,RT @KevGBKNY I think that?s out your system 2-...,think system days day use lot listerine peroxide,1
1,12/30/2018,Listerine,floss products,review,"I love these strips, but have had trouble find...",love strips trouble finding area please tell s...,1


In [64]:
brandwatch_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240337 entries, 0 to 240336
Data columns (total 7 columns):
Date                240337 non-null object
Brand               240337 non-null object
Product_category    74297 non-null object
Source              240294 non-null object
Review_original     240337 non-null object
Review_clean        240337 non-null object
Sentiment           240337 non-null int64
dtypes: int64(1), object(6)
memory usage: 12.8+ MB


In [65]:
# export to csv
brandwatch_clean.to_csv('Brandwatch_Clean.csv')

# Combine Brandwatch and BazarVoice Datasets

In [72]:
# bring in clean bazar voice 
bazar_clean = pd.read_csv("Bazar_Clean.csv", header = 0, engine='python')

In [73]:
# Combine the two datasets
All_Clean = pd.concat([bazar_clean, brandwatch_clean],axis=0)
All_Clean.info()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 248928 entries, 0 to 240336
Data columns (total 8 columns):
Brand               248928 non-null object
Date                248928 non-null object
Product_category    82888 non-null object
Review_clean        248927 non-null object
Review_original     248928 non-null object
Sentiment           248928 non-null int64
Source              248885 non-null object
Unnamed: 0          8591 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 17.1+ MB


In [74]:
All_Clean = All_Clean.sort_values(by=['Date'])
All_Clean

,Brand,Date,Product_category,Review_clean,Review_original,Sentiment,Source,Unnamed: 0
29682,Listerine,1/1/2018,tartar control mouthwash collection,listerine could ever use worstera bestbreath,RT @KeithOlbermann But...all the Listerine the...,1,twitter,NaN
29679,Listerine,1/1/2018,zero alcohol-free mouthwash collection,recommended dental hygienist burn mouth tastes...,Recommended by my dental hygienist. Doesn't bu...,1,review,NaN
29678,Listerine,1/1/2018,floss products,similar example think red shoe sole louboutin ...,The most similar example I can think of is the...,1,reddit,NaN
29677,Listerine,1/1/2018,zero alcohol-free mouthwash collection,gargle really warm salt water hit enough burn ...,@Zellyanks Gargle with really warm salt water....,1,twitter,NaN
29676,Listerine,1/1/2018,sensitivity,lice hate smell tea tree oil lice shampoo easi...,Lice hate the smell of tea tree oil. The lice ...,1,forum,NaN
...,...,...,...,...,...,...,...,...
68833,Therabreath,9/9/2019,NaN,worked two days,Worked after two days,1,review,NaN
68834,Therabreath,9/9/2019,NaN,format health beauty know original product hop...,Format: Health and Beauty. I don't know what t...,1,review,NaN
68835,Therabreath,9/9/2019,NaN,problem earlier first trimester early second t...,"I had this problem earlier, during first trime...",1,forum,NaN
68826,Toms,9/9/2019,NaN,flavor orange mango anyone sensitive mouth bra...,Flavor: Orange-Mango. For ANYONE with a sensit...,1,review,NaN


In [75]:
All_Clean["Product_category"].value_counts()

on-the-go oral care products                                   18481
antiseptic mouthwash collection                                17892
nightly reset                                                  12140
sensitivity                                                     9314
zero alcohol-free mouthwash collection                          8777
floss products                                                  4994
tartar control mouthwash collection                             4652
kids mouthwash collection                                       3127
fluoride toothpaste collection                                  1127
total care mouthwash collection                                 1099
naturals mouthwash collection                                    757
ultraclean tartar control mouthwash                              343
healthy whiteãƒâ¢?ã‚â¢ teeth whitening mouthwash collection      157
healthy whiteÃ¢?Â¢ teeth whitening mouthwash collection           26
fluoride defenseÃ¢?Â¢             

In [80]:
# export to csv
All_Clean.to_csv('All_Clean.csv')

In [78]:
# ID Retweet Reviews and create new df that does not contain these duplicates. 
All_Clean.Review_original=All_Clean.Review_original.astype(str)
All_Clean_noRT = All_Clean[~All_Clean.Review_original.str.contains("RT", na=False)]
All_Clean_noRT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199415 entries, 29679 to 69230
Data columns (total 8 columns):
Brand               199415 non-null object
Date                199415 non-null object
Product_category    63757 non-null object
Review_clean        199414 non-null object
Review_original     199415 non-null object
Sentiment           199415 non-null int64
Source              199372 non-null object
Unnamed: 0          8580 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 13.7+ MB


In [4]:
All_Clean_noRT= pd.read_csv('All_Clean_noRT.csv', header = 0, engine='python')

In [5]:
All_Clean_noRT['Source'].value_counts()

twitter          84189
review           51318
forum            18062
news             12749
reddit           12666
tumblr           10957
listerine.com     8580
blog               843
Name: Source, dtype: int64

In [79]:
# export no Retweet (RT) file to csv
All_Clean_noRT.to_csv('All_Clean_noRT.csv')